We want to design a box of height $h$, width $w$, and depth $d$ with maximum volume $(hwd)$ subject to the limits:

* total wall area: $2(hw + hd) \le A_{wall}$
* total floor area: $wd \le A_{flr}$
* height-width aspect ratio: $\alpha \le \frac{h}{w} \le \beta$
* width-depth aspect ratio: $\gamma \le \frac{d}{w} \le \delta$

In [1]:
Aw = 8
Af = 2
α = 0.25
β = 4
γ = 0.5
δ = 2


2

Here is an optimization model 

\begin{aligned}
\min_{h,w,d\,>\,0}\quad& h^{-1}w^{-1}d^{-1} \\
\text{subject to:}\quad& \tfrac{2}{A_{wall}} hw + \tfrac{2}{A_{wall}} hd \le 1,&&\tfrac{1}{A_{flr}} wd \le 1 \\
&\alpha h^{-1} w \le 1,&& \tfrac{1}{\beta}h w^{-1} \le 1 \\
&\gamma w d^{-1} \le 1,&& \tfrac{1}{\delta}w^{-1} d \le 1
\end{aligned}

This looks like it has nonconvex functions in it, but if we define 
$x := \log h$, $y := \log w$, and $z := \log d$, it gets transformed into the following: 

\begin{aligned}
\min_{x,y,z}\quad& -x-y-z \\
\text{subject to:}\quad& \log( e^{\log(2/A_{wall}) + x + y} + e^{\log(2/A_{wall}) + x+z}) \le 0 \\
&  y + z \le \log A_{flr} \\
& \log\alpha \le x - y \le \log\beta \\
& \log\gamma \le z- y  \le \log\delta 
\end{aligned}

In [2]:
using JuMP, SCS 

model = Model(SCS.Optimizer)
set_silent(model)
@variable(model, x)
@variable(model, y)
@variable(model, z)
@variable(model, u1)
@variable(model, u2)

@objective(model, Max, x + y + z)
@constraint(model, [log(2/Af) + x + y, 1, u1] in MOI.ExponentialCone())
@constraint(model, [log(2/Af) + x + z, 1, u2] in MOI.ExponentialCone())
@constraint(model, u1 + u2 <= 1)

@constraint(model, y+z <= log(Af))
@constraint(model, log(α) <= x - y <= log(β))
@constraint(model, log(γ) <= z - y <= log(δ))

optimize!(model)
if termination_status(model) == MOI.OPTIMAL && primal_status(model) == MOI.FEASIBLE_POINT
    println("The model is solved and feasible.")
else
    println("The model is not solved or not feasible.")
end

#print(objective_value(model))

# Transform it back and see shape
h = exp(value(x))
w = exp(value(y))
d = exp(value(z))

print("Box has dimensions (h,w,d) = (", round(h,digits=2), ",", 
round(w,digits=2), ",",  round(d,digits=2), ")")


#print(model)




The model is solved and feasible.
Box has dimensions (h,w,d) = (0.35,1.41,1.41)